# 6 RNN for language modeling

In [17]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io

/opt/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
text = open('shakespeare.txt', 'r').read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 5342761
total chars: 54


In [19]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in tqdm(range(0, len(text) - maxlen, step)):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

100%|██████████| 1780907/1780907 [00:01<00:00, 979978.46it/s] 

nb sequences: 1780907


In [20]:
# Vectorization
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

1780907it [00:27, 64377.77it/s]


In [21]:
X.shape

(1780907, 40, 54)

In [22]:
y.shape

(1780907, 54)

In [23]:
# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               93696     
_________________________________________________________________
dense_1 (Dense)              (None, 54)                6966      
_________________________________________________________________
activation_1 (Activation)    (None, 54)                0         
Total params: 100,662
Trainable params: 100,662
Non-trainable params: 0
_________________________________________________________________


In [24]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [25]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(200):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [27]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(X, y,
          batch_size=4096,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
1780907/1780907 [==============================] - 69s 38us/step - loss: 2.1090 - acc: 0.3829

----- Generating text after Epoch: 0
----- Generating with seed: "w!
	that pure congealed white, high taur"
w!
	that pure congealed white, high taurs to tith.

casunol	again, i by all, befied.

flord	this have this ever hiadtencud,
	i be could reath, shall must forty me; to detwo
	deeptheds stret elone wonds, aw you imop of a boge:
	gimative, how
Epoch 2/60
1780907/1780907 [==============================] - 63s 35us/step - loss: 1.6357 - acc: 0.5039

----- Generating text after Epoch: 1
----- Generating with seed: "wn her consecrated wall,
and by their mo"
wn her consecrated wall,
and by their most up your yiched hand and see-but curming a pliused time,
	at-aspies of but be drem, 'tis no more piege;
	no were in this recrest on appearus
	lain aim quoth no lovent pard'd aymen.

calumand	how no 
Epoch 3/60
1780907/1780907 [==============================] - 61s 34us/step - loss: 1.5333

/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


upon him?

dromio orialice	yet nurs. master, let me came your spite to leave him to wherelous,
	are-virghace. little stone, that knock'd one perperercable: only this call
	
Epoch 12/60
1780907/1780907 [==============================] - 60s 34us/step - loss: 1.3807 - acc: 0.5706

----- Generating text after Epoch: 11
----- Generating with seed: "
	down indirectly to the jew's house.

g"

	down indirectly to the jew's house.

gloucester	i pay the woels' quiet heralt. give me noble
	be roman
	simplingession the tailors are in;
	commands for face. hear my heart than thou,
	so, would weep them and meedles? this know
	this moon
Epoch 13/60
1780907/1780907 [==============================] - 60s 34us/step - loss: 1.3758 - acc: 0.5722

----- Generating text after Epoch: 12
----- Generating with seed: "n the banquet quickly; wine enough
	cleo"
n the banquet quickly; wine enough
	cleon much respectus
hadly-bidful	a sin. was he repent these of a foos all'd the
	john of the women or dived of those 

	when you are engling hath money doth
	priestery of grave to beaten his drops,
	this strait that but unnothy not all
	many and been furw. at flies his orst more and
	heart you with an other indireus, that in
	th
Epoch 30/60
1780907/1780907 [==============================] - 61s 34us/step - loss: 1.3367 - acc: 0.5817

----- Generating text after Epoch: 29
----- Generating with seed: "r accommodate
	his master thus.

king le"
r accommodate
	his master thus.

king lear	in up, occas, what mons.

troilus	go, no fortune, then fly as't part, and larguland.
	why should say, for my as should it, good man. i think it,
	speak with hand. and, my liege, i troinity.

prince
Epoch 31/60
1780907/1780907 [==============================] - 60s 34us/step - loss: 1.3355 - acc: 0.5820

----- Generating text after Epoch: 30
----- Generating with seed: " are paid too much; purse and brain
	bot"
 are paid too much; purse and brain
	both a mairle reverend marriage her matter
	else we
differed	meder of it.

	[f

	and had to subjects the repelle of death,
pence would leave the time will churchs the nozen has love at
	me neigh him out of these threater hall feed
	a particularies and the modes resamon,
  like even
Epoch 48/60
1780907/1780907 [==============================] - 57s 32us/step - loss: 1.3222 - acc: 0.5852

----- Generating text after Epoch: 47
----- Generating with seed: "ase me with speaking to me, thou
	mights"
ase me with speaking to me, thou
	mights from slow of that you corrury! to thee full,
	and deep dicking his own state of her slendon,
	less call and preisor'd your lord care are a boot.

	[exeunt all is a since medice, like thou custest:
	i
Epoch 49/60
1780907/1780907 [==============================] - 57s 32us/step - loss: 1.3216 - acc: 0.5854

----- Generating text after Epoch: 48
----- Generating with seed: " say,
	one that takes upon him to be a d"
 say,
	one that takes upon him to be a dead
	to bectors., tolking
	a mysext still be wit them.' if canst wend? pray you,
	co

In [32]:
start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1, 1.2]:
    print()
    print('----t = %f----' % diversity)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    print(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----t = 0.200000----
----- Generating with seed: "gh flame, and
	through ford and whirlipo"
gh flame, and
	through ford and whirlipo
rs of the strings.


/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


	thou hast married the court of the strength
	and this that i shall be so the devil.

	[enter a gentleman]

	why, thou art the suppose of the court and the strength
	to the court of the strange of the strings,
	and the strings and should be so things to the
	force, the brother of the prince of the stranger
	the strength and the strings of the love
	that i will be the strings of 

----t = 0.500000----
----- Generating with seed: "gh flame, and
	through ford and whirlipo"
gh flame, and
	through ford and whirlipo
us souls be a company
	to mercy and therefore the king conceived,
	and the store of her thing shall be my sport.

	[enter a cousin]

	and i cannot be sent the earth, the part
	is to the strange of a wit to the devil,
	and stand the sent of the stranger to the world.
	the strength and a say the sea of the world.

lord anconero	the part, and then be warrant and again.

	[enter sir john of donm call 

----t = 1.000000----
----- Generating with seed: "gh flame, and
	through ford and 

In [47]:
model.save('model_lm.dat')